# Deliverable

* Correlate each column with S&P Data Total Return (which can be pulled from online somewhere) and the price return (without dividend
* How do the past activity of all of those combined correlate with future activities (3, 6, 12 month time periods) with the S&P price return

# Libraries

In [2]:
import pandas as pd
import numpy as np

# Assumptions

Assumptions I make will be listed below. 
* Used **yahoo finance data** for S&P 500 prices - https://finance.yahoo.com/quote/%5EGSPC/history?p=%255EGSPC&nn=1 (used the API, as this is not available for download by conventional users)


# Plan

1. Read in CSV from Excel 
    * Clean date into datetime
2. Find a S&P 500 Price Return (no didend included in return) from online, list source in assumptions so it can be verified
3. Add a column with S&P price return 
    * The intervals I will use are 90 day, 180 day, and 360 day (~3 month, ~6 month, ~12 month intervals)
    * Close to close values are used ()

# 1 - Reading in CSV

In [3]:
data = pd.read_csv('./fsi.csv')
data.head()

,Date,OFR FSI,Credit,Equity valuation,Safe assets,Funding,Volatility,United States,Other advanced economies,Emerging markets
0,1/3/2000,2.140,0.540,-0.051,0.670,0.472,0.509,1.769,0.521,-0.150
1,1/4/2000,2.421,0.604,0.079,0.627,0.550,0.561,2.084,0.474,-0.137
2,1/5/2000,2.297,0.575,0.080,0.653,0.501,0.488,2.023,0.406,-0.132
3,1/6/2000,2.292,0.546,0.082,0.581,0.566,0.517,2.064,0.355,-0.127
4,1/7/2000,2.005,0.662,-0.011,0.519,0.565,0.270,1.769,0.367,-0.131


In [5]:
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y') 
data.head()

,Date,OFR FSI,Credit,Equity valuation,Safe assets,Funding,Volatility,United States,Other advanced economies,Emerging markets
0,2000-01-03,2.140,0.540,-0.051,0.670,0.472,0.509,1.769,0.521,-0.150
1,2000-01-04,2.421,0.604,0.079,0.627,0.550,0.561,2.084,0.474,-0.137
2,2000-01-05,2.297,0.575,0.080,0.653,0.501,0.488,2.023,0.406,-0.132
3,2000-01-06,2.292,0.546,0.082,0.581,0.566,0.517,2.064,0.355,-0.127
4,2000-01-07,2.005,0.662,-0.011,0.519,0.565,0.270,1.769,0.367,-0.131


In [33]:
data.tail()

,Date,OFR FSI,Credit,Equity valuation,Safe assets,Funding,Volatility,United States,Other advanced economies,Emerging markets
5874,2023-04-25,-0.349,0.218,-0.118,-0.092,-0.533,0.176,-0.094,-0.421,0.166
5875,2023-04-26,-0.259,0.233,-0.096,-0.104,-0.491,0.199,-0.075,-0.346,0.162
5876,2023-04-27,-0.669,0.191,-0.145,-0.131,-0.645,0.061,-0.342,-0.474,0.147
5877,2023-04-28,-0.682,0.192,-0.182,-0.117,-0.516,-0.059,-0.317,-0.499,0.134
5878,2023-05-01,-0.694,0.174,-0.174,-0.151,-0.502,-0.041,-0.326,-0.482,0.114


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5879 entries, 0 to 5878
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      5879 non-null   object 
 1   OFR FSI                   5879 non-null   float64
 2   Credit                    5879 non-null   float64
 3   Equity valuation          5879 non-null   float64
 4   Safe assets               5879 non-null   float64
 5   Funding                   5879 non-null   float64
 6   Volatility                5879 non-null   float64
 7   United States             5879 non-null   float64
 8   Other advanced economies  5879 non-null   float64
 9   Emerging markets          5879 non-null   float64
dtypes: float64(9), object(1)
memory usage: 459.4+ KB


In [26]:
from pandas_datareader import data as pdr
import datetime as dt
import yfinance as yfin 
yfin.pdr_override()

start_date = dt.datetime(2001, 1, 1)
end_date = dt.datetime(2023, 5, 2)

df = pdr.get_data_yahoo("^GSPC", start="2000-01-03", end="2023-05-02")
df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-25,4126.430176,4126.430176,4071.379883,4071.629883,4071.629883,3978640000
2023-04-26,4087.780029,4089.669922,4049.350098,4055.989990,4055.989990,3837030000
2023-04-27,4075.290039,4138.240234,4075.290039,4135.350098,4135.350098,3750550000
2023-04-28,4129.629883,4170.060059,4127.180176,4169.479980,4169.479980,4087800000
2023-05-01,4166.790039,4186.919922,4164.120117,4167.870117,4167.870117,3321370000


In [28]:
df["3MonPriceReturn"] = df["Close"].shift(-90) - df["Close"]
df["6MonPriceReturn"] = df["Close"].shift(-180) - df["Close"]
df["12MonPriceReturn"] = df["Close"].shift(-360) - df["Close"]
df.head()

,Open,High,Low,Close,Adj Close,Volume,3MonPriceReturn,6MonPriceReturn,12MonPriceReturn
Date,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,-47.409912,4.680054,-178.260010
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,21.539917,51.919922,-134.460083
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,50.250000,46.940063,-147.719971
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,62.590088,45.270020,-147.599976
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,6.330078,-2.439941,-199.869995


In [29]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,3MonPriceReturn,6MonPriceReturn,12MonPriceReturn
Date,,,,,,,,,
2023-04-25,4126.430176,4126.430176,4071.379883,4071.629883,4071.629883,3978640000,NaN,NaN,NaN
2023-04-26,4087.780029,4089.669922,4049.350098,4055.989990,4055.989990,3837030000,NaN,NaN,NaN
2023-04-27,4075.290039,4138.240234,4075.290039,4135.350098,4135.350098,3750550000,NaN,NaN,NaN
2023-04-28,4129.629883,4170.060059,4127.180176,4169.479980,4169.479980,4087800000,NaN,NaN,NaN
2023-05-01,4166.790039,4186.919922,4164.120117,4167.870117,4167.870117,3321370000,NaN,NaN,NaN


It is expected that the last 90 rows of 3 Month Price Return, 180 of 6 Month Price Return, and last 360 of 12 Month Price Return will be NULL (NaN)

In [31]:
df.shape

(5869, 9)

In [32]:
data.shape

(5879, 10)